In [1]:
%reload_ext autoreload
%autoreload 2
from pathlib import Path
from glob import glob
import pandas as pd
import sys
sys.path.append(str(Path.cwd().parent))

# Get Data
You will find $^{18}\text{FDG-PET}$ data at: https://osf.io/t753c/files/osfstorage/696dfc20b9533d640ae29c68 \
as well as $\text{MEG}$ data at: https://osf.io/t753c/files/osfstorage/696dfbd547d945faeaba3393

Put PET data and decompress MEG data at working folder

In [ ]:
ad = glob("./lorentzian_ad_data/*ad.npz")
cn = glob("./lorentzian_ad_data/*cn.npz")

# Compute PSD param

In [45]:
from optimized_pipeline import sequential_fit, _process_one, compute_params_parallel
import numpy as np
from excitability_fit import spectral_exponent_per_channel
from tqdm.notebook import tqdm

In [ ]:
lorentzian_dfs = []
excitability_dfs = []
f = None
for i, file in enumerate(tqdm(ad + cn, desc="Processing PSDs")):
    data = np.load(file)
    f = data["f"]
    psds = data["psds"]
    labels = data["labels"]
    subject = data["id"]
    group = data["group"]
    lorentzian_df = compute_params_parallel(psds, f)
    lorentzian_df["brainnetome_index"] = labels
    lorentzian_df["group"] = group
    lorentzian_df["subject"] = subject
    lorentzian_dfs.append(lorentzian_df)

    excitability_df = spectral_exponent_per_channel(
        freqs=f, psds=psds,
        fmin=15.0, fmax=50.0
    )
    excitability_df["brainnetome_index"] = labels
    excitability_df["group"] = group
    excitability_df["subject"] = subject
    excitability_dfs.append(excitability_df)

In [48]:
l_merged_dfs = pd.concat(lorentzian_dfs, ignore_index=True)
e_merged_dfs = pd.concat(excitability_dfs, ignore_index=True)
fdg_data = pd.read_csv("fdg_data.csv")
fdg_data["subject"] = fdg_data["subject"].astype(int)
l_merged_dfs["subject"] = l_merged_dfs["subject"].astype(int)
e_merged_dfs["subject"] = e_merged_dfs["subject"].astype(int)
df = l_merged_dfs.merge(
    e_merged_dfs,
    on=["brainnetome_index", "subject", "group"],
    how="left"
).merge(
    fdg_data,
    on=["subject", "brainnetome_index"],
    how="left"
)

# LMMs

In [49]:
def compute_z_score(
    df: pd.DataFrame,
    value_col: str,
    group_col: str,
    ref_label: str = "cn",
    prefix: str = "z_",
    index: str = "brainnetome_index",
):
    ref = df[df[group_col] == ref_label]

    # Compute mean/std => Z per region on reference population
    means = ref.groupby(index)[value_col].mean()
    stds  = ref.groupby(index)[value_col].std()
    mu  = df[index].map(means)
    sig = df[index].map(stds)
    df[prefix + value_col] = (df[value_col] - mu) / sig

    return df

for label in ["slope", "l_exp", "pl_exp", "l_fc", "suvr_fdg"]:
    compute_z_score(df, value_col=label, group_col="group")

## All models

In [ ]:
import statsmodels.formula.api as smf

results = {}
for y, x in zip(["z_suvr_fdg", "z_suvr_fdg", "z_suvr_fdg", "z_suvr_fdg"], ["z_l_fc", "z_l_exp", "z_pl_exp", "z_slope"]):
    sub_df = df[(df["brainnetome_index"] < 211) & (df["group"] == "ad")].copy() # Exclude subcortical regions and target AD group
    sub_df = sub_df.dropna(subset=[x, y, "age"]) # Drop NaNs in x, y, and age
    sub_df["age_c"] = sub_df["age"] - sub_df["age"].mean() # Center age variable
    model = smf.mixedlm(f"{y} ~ {x} + age_c", sub_df, groups=sub_df["subject"], re_formula=f"~ {x}") # Fit mixed effects model
    res = model.fit()
    results[x] = {
        "model": res,
        "df": sub_df
    }

In [83]:
from utils import summarize_mixedlm, fmt_p

rows = []
for x, d in results.items():
    res = d["model"]
    rows.append(summarize_mixedlm(res, model_name=x))

summary = pd.concat(rows, ignore_index=True)
fixed_terms = {"Intercept", "age_c"} | set(results.keys())
fixed_summary = summary[summary["term"].isin(fixed_terms)].copy()

display_df = fixed_summary.copy()
for c in ["coef", "se", "stat", "ci_low", "ci_high", "llf", "aic", "bic"]:
    display_df[c] = display_df[c].map(lambda v: "" if pd.isna(v) else f"{v:.3f}")
display_df["p"] = fixed_summary["p"].map(fmt_p)

display_df = display_df[[
    "model", "term", "coef", "se", "stat", "p", "ci_low", "ci_high",
    "n_obs", "n_groups", "llf", "aic", "bic"
]].loc[np.isin(display_df["term"], ["z_l_fc", "z_l_exp", "z_pl_exp", "z_slope"])]

display_df

,model,term,coef,se,stat,p,ci_low,ci_high,n_obs,n_groups,llf,aic,bic
1,z_l_fc,z_l_fc,0.417,0.083,4.997,<1e-4,0.254,0.581,9028,43,-13037.692,,
7,z_l_exp,z_l_exp,-0.315,0.078,-4.056,<1e-4,-0.468,-0.163,9028,43,-13252.067,,
13,z_pl_exp,z_pl_exp,0.007,0.023,0.319,0.7500,-0.038,0.053,8460,43,-12567.924,,
19,z_slope,z_slope,-0.392,0.091,-4.329,<1e-4,-0.570,-0.215,9028,43,-13009.656,,


## Slope is Lorentzian cutoff frequency + Lorentzian b

In [59]:
import statsmodels.formula.api as smf

y_plan = "z_slope"
x1_plan = "z_l_fc"
x2_plan = "z_l_exp"
sub_df = df[(df["brainnetome_index"] < 211)].copy()
sub_df = sub_df.dropna(subset=[x1_plan, x2_plan, y_plan])
model = smf.mixedlm(
    f"{y_plan} ~ {x1_plan} + {x2_plan}",
    sub_df,
    groups=sub_df["subject"],
    re_formula="~1"  # random intercept only
)
res = model.fit()

names = [x1_plan, x2_plan]
betas = res.fe_params[names]
ses = res.bse[names]
print("highest pvalue:", res.pvalues["z_l_fc"])
res.summary().tables[1]

highest pvalue: 1.1432416115418962e-241


,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
Intercept,-0.004,0.051,-0.077,0.938,-0.104,0.096
z_l_fc,0.244,0.007,33.198,0.000,0.229,0.258
z_l_exp,0.773,0.008,99.025,0.000,0.757,0.788
Group Var,0.176,0.080,,,,


# Full Lorentzian model explains more than excitability index

In [53]:
from scipy import stats
sub_df = df[df["brainnetome_index"] < 211].copy()
sub_df = sub_df.dropna(subset=["z_suvr_fdg", "z_slope", "age", "z_l_fc", "z_l_exp"])
m_slope = smf.mixedlm(f"z_suvr_fdg ~ z_slope + age", sub_df, groups=sub_df["subject"], re_formula = f"~ z_slope").fit(reml=False, method="lbfgs")
m_lorentzian = smf.mixedlm(f"z_suvr_fdg ~ z_l_fc + z_l_exp + age", sub_df, groups=sub_df["subject"], re_formula = f"~ z_l_fc + z_l_exp").fit(reml=False, method="lbfgs")
aic_slope = m_slope.aic
aic_lor   = m_lorentzian.aic

delta_aic = aic_slope - aic_lor
rel_likelihood = np.exp(-abs(delta_aic) / 2)
pd.DataFrame({
    "aic_slope": [aic_slope],
    "aic_lorentzian": [aic_lor],
    "delta_aic": [delta_aic],
    "rel_likelihood": [rel_likelihood]
}).round(3)

,aic_slope,aic_lorentzian,delta_aic,rel_likelihood
0,26009.236,25763.0,246.235,0.0
